
Задание 1.
Обязательная часть

Будем парсить страницу со свежими новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>



In [71]:
KEYWORDS = ['design', 'java']

In [162]:
import requests
import json

In [73]:
from bs4 import BeautifulSoup

In [74]:
import time

In [81]:
import pandas as pd

In [75]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [213]:
posts = soup.find_all('article', class_='post')


In [170]:
habr_news = pd.DataFrame()
findings_list = []
for p in posts:
    findings = p.find('div', class_='post__text post__text-html post__text_v1')
    if findings is None:
        findings = p.find('div', class_='post__text post__text-html post__text_v2')
    for k in KEYWORDS:
        if k in findings.text.lower():
            date = p.find('span', class_='post__time').text
            title = p.find('h2', class_='post__title').text.strip()
            link = p.find('a', class_='post__title_link').get('href')
            article = BeautifulSoup(requests.get(link).text, 'html.parser')
            text = article.find('div', class_='post__text post__text-html post__text_v1').text.strip()
            if text is None:
                text = article.find('div', class_='post__text post__text-html post__text_v2').text.strip()
            row = {'date': date, 'title': title, 'link': link, 'text': text}
            habr_news = pd.concat([habr_news, pd.DataFrame([row])])
            
habr_news

,date,title,link,text
0,сегодня в 12:52,Motion Design в Dota 2,https://habr.com/ru/post/517354/,"Друзья, меня зовут Артём и хочу рассказать вам..."
0,сегодня в 11:20,JavaScript: область видимости простыми словами,https://habr.com/ru/post/517338/,"Доброго времени суток, друзья!\n\r\nОбласть ви..."
0,сегодня в 10:00,"Awesome-лист своими руками, или GitHub вместо ...",https://habr.com/ru/company/croc/blog/496594/,"Привет, Хабр! Наверное, у каждого из нас есть ..."



Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.


In [158]:
EMAIL = ['vasya@ya.ru', 'vasya@gmail.com']

In [171]:
def check_breaches(emails):
    breaches = pd.DataFrame()
    URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
    headers = {'Vaar-Version': '0', 'Vaar-Header-App-Product': 'hackcheck-web-avast'}
    for e in EMAIL:
        req = requests.post(URL, headers=headers,json={"emailAddresses":[e]})
        for v in req.json()['breaches'].values():
            breaches_dict = {'e-mail': e, 'date': v['publishDate'], 'source': v['site'], 'description': v['description']}
            breaches = pd.concat([breaches, pd.DataFrame([breaches_dict])])
    return breaches

In [172]:
check_breaches(EMAIL)

,e-mail,date,source,description
0,vasya@ya.ru,2016-10-26T00:00:00Z,twitter.com,Login credentials for over 32 Million Twitter ...
0,vasya@ya.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
0,vasya@ya.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,vasya@ya.ru,2016-11-01T00:00:00Z,qip.ru,"In 2011, Russian instant messaging service pro..."
0,vasya@ya.ru,2018-06-19T00:00:00Z,storelp.ru,"At an unconfirmed date, StoreLP.ru's database ..."
...,...,...,...,...
0,vasya@gmail.com,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,vasya@gmail.com,2018-02-03T00:00:00Z,lsgb.net,"On an unconfirmed date, the manufacturing comp..."
0,vasya@gmail.com,2017-03-31T00:00:00Z,fc-zenit.ru,"In July 2010, FC Zenit's user database was all..."
0,vasya@gmail.com,2018-07-10T00:00:00Z,lpfiction.com,"At an unconfirmed date, LPfiction's database w..."



Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.


In [211]:
def get_group_posts(group):
    WALL_REQUEST = 'https://api.vk.com/method/wall.get?'
    TOKEN = '...'
    VERSION = '5.21'
    SLEEP = 0.33
    params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': group,
    'count': 50
    }
    vk_group = requests.get(WALL_REQUEST, params)
    data = pd.DataFrame(vk_group.json()['response']['items'])[['date', 'text']]
    return data

In [212]:
get_group_posts('netology')

,date,text
0,1598629320,"Необязательно дожидаться 1 сентября, чтобы пол..."
1,1598972460,Новый учебный год — хороший повод пересмотреть...
2,1598952602,Сегодня прозвенел первый звонок (но и он для у...
3,1598888880,Каким обычно представляют офис IT-компании? См...
4,1598859000,🗓8 сентября стартует бесплатный онлайн-интенси...
5,1598787720,Поиск работы или смена профессии — большой шаг...
6,1598710500,Сфера диджитал постоянно расширяется и нуждает...
7,1598688720,"Собрали подборку статей, которая заставит вас ..."
8,1598600520,Актуальная задачка для отпускников: давайте по...
9,1598542920,Исследования показывают: мы лучше запоминаем т...
